In [41]:
import pandas as pd
import numpy as np
import geopandas as gpd
from bokeh.plotting import figure, save
import os
# IMPORTANT NOTE: having difficulting writing the crs to shp using geopands to_file
# Can do it manually using notepad++ though. WORK IN PROGRESS
from shapely.geometry import Point
import matplotlib.pyplot as plt
%matplotlib inline
import janmaps as jm # personal functions for mapping
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper
import pysal as ps

import plotly
import plotly.plotly as py
plotly.tools.set_credentials_file(username='XXX',api_key='XXX')
from plotly.graph_objs import *
plotly.offline.init_notebook_mode(connected=True)

In [42]:
p = figure(title = 'My first interactive plot')
AWU = gpd.read_file(r'D:\Documents\ML\Geo\shapefiles\ServiceArea\AWUServiceArea.shp')
zpco = gpd.read_file(r'D:\Documents\ML\Geo\shapefiles\zipcodes\Austinzipcodes.shp')
df = pd.read_excel(r'D:\Documents\ML\Geo\Leaks_forCo.xlsx',sheet_name='Sheet1')

In [43]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [44]:
zpco['x'] = zpco.apply(getPolyCoords, geom = 'geometry', coord_type = 'x', axis =1)
zpco['y'] = zpco.apply(getPolyCoords, geom = 'geometry', coord_type = 'y', axis =1)

In [45]:
zpco.head(5)

,objectid,name,shape_len,zipcodes_i,modified_d,created_da,modified_b,created_by,geodb_oid,shape_area,zipcode,geometry,x,y
0,1.0,AUSTIN,98511.305637,54.0,None,None,None,None,1.0,3.691369e+08,78754,"POLYGON ((-97.63657664958002 30.4026577780347,...","[-97.63657664958002, -97.63753810238059, -97.6...","[30.4026577780347, 30.398205247657316, 30.3981..."
1,2.0,GRANGER,300079.664111,55.0,None,None,None,None,2.0,2.916192e+09,76530,POLYGON ((-97.25315200612187 30.69313100882735...,"[-97.25315200612187, -97.25995900177988, -97.2...","[30.693131008827347, 30.692512020186516, 30.69..."
2,3.0,AUSTIN,97785.793334,56.0,None,None,None,None,3.0,3.383405e+08,78739,POLYGON ((-97.89001573769271 30.20941197333968...,"[-97.8900157376927, -97.889286257776, -97.8886...","[30.20941197333968, 30.208707741357834, 30.207..."
3,4.0,LEANDER,196654.442800,57.0,None,None,None,None,4.0,1.043396e+09,78645,"POLYGON ((-97.91074824923403 30.5010967622175,...","[-97.91074824923403, -97.91053037500839, -97.9...","[30.501096762217504, 30.50107887431547, 30.501..."
4,5.0,AUSTIN,121428.596913,58.0,None,None,None,None,5.0,4.792590e+08,78732,POLYGON ((-97.87302802996035 30.43806372004226...,"[-97.87302802996035, -97.8733460292974, -97.87...","[30.438063720042262, 30.436983423346792, 30.43..."


In [46]:
z_df = zpco.drop('geometry',axis=1).copy()
zsource = ColumnDataSource(z_df)

In [47]:
coordX = df.X
coordY = df.Y

# Create geodataframe from data
pts_geometry = [Point(xy) for xy in zip(coordX, coordY)]
pts_crs = {'init':'epsg:3857'}
points = gpd.GeoDataFrame(df, crs=pts_crs, geometry=pts_geometry)

In [49]:
CRS = zpco.crs
print(CRS)

{'init': 'epsg:4326'}


In [36]:
zpco['geometry'] = zpco['geometry'].to_crs(crs = CRS)

In [50]:
from bokeh.palettes import RdYlBu11 as palette
from bokeh.models import LogColorMapper

# Create the color mapper
color_mapper = LogColorMapper(palette=palette)

In [30]:
p = figure(title="Zipcode")

# Plot grid
p.patches('x', 'y', source=zsource,
                  fill_alpha=1.0, line_color="black", line_width=0.5)

# Add metro on top of the same figure
#p.multi_line('x', 'y', source=msource, color="red", line_width=2)

# Add points on top (as black points)
p.circle('x','y',source = psource, size=1, color="red")

# Save the figure
outfp = r'D:\Documents\ML\Geo\plotforall1.html'
save(obj = p, filename = outfp)

C:\Users\ASUS\Anaconda3\lib\site-packages\bokeh\io\saving.py:125: UserWarning:

save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN

C:\Users\ASUS\Anaconda3\lib\site-packages\bokeh\io\saving.py:138: UserWarning:

save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'



'D:\\Documents\\ML\\Geo\\plotforall1.html'

In [51]:
points

,HomeNo,SubAddress1,StName,SubAddress2,SubAddress3,City,ZIP,AddressID,MapZone,ObjectID,...,LeakRepairedTime,LeakRepairedTime2,DaysElapsed,Week,Month,Year,Bi-Week,DateOnly,FullStName,geometry
0,9605,N,FM 973,RD,NaN,AUSTIN,78653,3374522,NaN,1038936,...,1475183160000,2016-09-29 21:06:00,NaN,40,9,2016,40,9/29/2016,9605 N FM 973 RD,POINT (-10860848.8235 3542441.27329999)
1,7304,NaN,LONGFORD,TRL,NaN,AUSTIN,78653,5092550,528G,1056782,...,1476421748847,2016-10-14 05:09:08,0.317464,42,10,2016,42,10/14/2016,7304 LONGFORD TRL,POINT (-10865000.6851 3549474.4441)
2,6413,NaN,BOYCE,LN,NaN,MANOR,78653,3047359,528B,1048143,...,1475816870934,2016-10-07 05:07:50,0.126284,41,10,2016,42,10/7/2016,6413 BOYCE LN,POINT (-10865699.4999 3550194.7531)
3,6413,NaN,BOYCE,LN,NaN,MANOR,78653,3047359,528B,1050326,...,1475896952854,2016-10-08 03:22:32,1.053158,41,10,2016,42,10/8/2016,6413 BOYCE LN,POINT (-10865699.4999 3550194.7531)
4,5600,NaN,WAR ADMIRAL,DR,NaN,DEL VALLE,78617,3049059,678P,1097471,...,1480307851468,2016-11-28 04:37:31,0.273281,49,11,2016,50,11/28/2016,5600 WAR ADMIRAL DR,POINT (-10865812.9825 3525692.1875)
5,9775,E,US 290 HWY,SVRD,2018-01-02 00:00:00,AUSTIN,78724,5077135,528X,1261617,...,1493188657071,2017-04-26 06:37:37,0.476818,17,4,2017,18,4/26/2017,9775 1/2 E US 290 HWY SVRD EB,POINT (-10865815.5598 3546548.6322)
6,16118,NaN,FM 812,RD,NaN,AUSTIN,78617,3372550,738P,1213275,...,1489131451918,2017-03-10 07:37:31,0.407314,10,3,2017,10,3/10/2017,16118 FM 812 RD,POINT (-10865883.3765 3516941.32859999)
7,9500,NaN,DECKER,LN,NaN,AUSTIN,78724,3050731,NaN,1169608,...,1485551640000,2017-01-27 21:14:00,NaN,4,1,2017,4,1/27/2017,9500 DECKER LN,POINT (-10865906.9717 3544933.22439999)
8,15001,NaN,JACOBSON,RD,NaN,DEL VALLE,78617,99534,NaN,1312146,...,1499711400000,2017-07-10 18:30:00,NaN,28,7,2017,28,7/10/2017,15001 JACOBSON RD,POINT (-10865926.1574 3520651.5318)
9,11920,NaN,DUNFRIES,LN,NaN,AUSTIN,78754,3127585,528F,1033915,...,1474760252967,2016-09-24 23:37:32,0.295520,39,9,2016,40,9/24/2016,11920 DUNFRIES LN,POINT (-10865932.9761 3549424.0451)


In [12]:
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

In [27]:
points['x'] = points.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
points['y'] = points.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)

In [28]:
p_df = points.drop('geometry',axis=1).copy()
psource = ColumnDataSource(p_df)